## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from scipy import stats

ImportError: cannot import name '_path'

In [2]:
data = pd.read_csv('credit_scoring_eng.csv')

#general information of the data
print(data.info())
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None



In [ ]:
#visual checking last or first values of the data
data.head(10)

In the interest of analyzing data for errors I decide to separate variable into two groups: categorical and numerical
categorical data - Descriptive data containing a small number of values inside - from 2 to 20 approximately values.
numerical - Quantitative characteristics of the client, for the analysis of which average measures are used

In [ ]:
#lets check numerical variables
print("Numerical variables:")
print()

#let's look of NAns present in var's where observations are less than in other variables
print('NA`s in variables:')
print(data[['total_income', 'days_employed']].isna().count())
print()

#obviouse problem was mentioned in variable 'days_employed'
#find the percent of the garbage data in problem variable 'days_emplyed'
print("- Percent of the garbage data in problem variable 'days_emplyed': {:.2%}".format(
    data['days_employed'][data['days_employed'] < 0].count() / data['days_employed'].count()))

#conseder other numeric vars "dob_years"
#let's see how many customers of different ages
#print(data['dob_years'].value_counts().sort_values(ascending=False))
#investigation shown that we have zero-age clients. This is an error. Lets count how much of them. 
print('- There are {} zero-age clients, and {} less-than-zero-age in "dob_years" variable'.format(
    data['dob_years'][data['dob_years'] == 0].count(), 
    data['dob_years'][data['dob_years'] < 0].count()))

#conseder other numeric vars "total_income"
print('- There are {} observations in total (it`s less than observations in the entire data becouse of NA`s), {} - zero-income clients, and {} less-than-zero-income in "total_income" variable (looks fine)'.format(
    data['total_income'].count(),
    data['total_income'][data['total_income'] == 0].count(),
    data['total_income'][data['total_income'] < 0].count()))
print()
#data in 'total_income' var looks fine

In [ ]:
#consider categorical variables separately
print("Categorical variables:")
print()

#print(data['children'].value_counts())
print('- There are {} observations in total, {} - 20 childrens, and {} -1 in "children" variable'.format(
    data['children'].count(),
    data['children'][data['children'] == 20].count(),
    data['children'][data['children'] < 0].count()))
print()

#print(data['education'].value_counts())
#print()
#seen case-sensitively duplicates 

#print(data['family_status'].value_counts())
#print()
#everything fine with this data 

#print(data['gender'].value_counts())
#print()
#seen statistically insignificant category "XNA" with one observation

#print(data['income_type'].value_counts())
#print()
#seen statistically insignificant 3 category "unemployed", "student" and "paternity / maternity leave" with less than two observation
#observation "entrepreneur" could be include to the "business" category

#print(data['debt'].value_counts())
#print()
#everything fine with this data 

#print(data['purpose'].value_counts())
#print()
#Seen a lot of different purposes. Looks like it needs unification in large groups.

### Conclusion

It seen that we have data about 21525 people. Data containces int, float and object tipes of the 12 variables. Column 'total_income' as 'days_employed' doesn't equal to the number of observation of the other variables in the data - 19351 vs 21525. Also data errors detected by visual evaluation in column 'days_employed'. Maybe data contains more errors like this. It would be seen after investigation in the next steps.

After a deeper study of the variables individually, a number of errors were identified. 

**Missing values**
Were identify 101 zero-age clients, and 0 less-than-zero-age in "dob_years" variable. "total_income" and "days_emplyed" variables has less observations that in the data 19351 vs 21525 becouse of NA's. 

**Data type replacement**
The days_employed variable in type 'float' must be replaced with type '.int' because day is an integer. 

**Processing duplicates**
In 'education' variable seen case-sensitively duplicates. Also in "children" variable discovered 76 observetion with 20 children and 47 with -1 children. It's abnormal values. It doesn't impossible that some of the clients has 20 children, but if that were true we would see some kind of normal distribution. But it isn't observed. Instead we see huge leap from 5 to 20 children. Highly possible that in clients have 2 children instead 20, and 1 instead -1. Additional symbols is the reason of the human factor. The variable 'days_emplyed' consist 82.2% nagative values. Negative values needs deeper investigation.

**Categorizing Data**
Investigation of 'gender' variable shown statistically insignificant category "XNA" with one observation. Also in 'income_type' var seen tree statistically insignificant categories "unemployed", "student" and "paternity / maternity leave" with less than two observations. Observation "entrepreneur" could be include to the "business" category. Insignificant categories could be reduce. In the last variable 'purpose' was seen a lot of different options of purposes. It seems that they need to be unify in larger groups. A lot of nagative values, in the variable 'days_emplyed', it mean that this variable couldn't be used for analises in this form. Maybe it make sense to divide them into two groups. And finally variables such as 'days_emplyed' and 'total_income' could by unified to more general groups too.

### Step 2. Data preprocessing

### Processing missing values

In [ ]:
#delete NA
data = data.fillna(value=0)

### Conclusion

Searching for the mistakes in the data, in general, I found of that two variable has less observations than others. I made an assumption that one’s has NA's. In the credit banking segment income and work experience factors has a critical meaning, so the present of NA's could relate to some personal reasons of the clients. Maybe they said unreliable info and then this info were delete from the base. I decided to replace NA with zero, because it provides an opportunity for the further stay analisis and making a segmentation. Filtering zeros is not a problem.

### Data type replacement

#### 'days_employed'

In [ ]:
#convert to int
data['days_employed'] = data['days_employed'].astype('int')
print(data.info())

### Conclusion

The days_employed variable in type 'float' must be replaced with type '.int' because day is an integer.
I used `.astype('int')` because it works for this data.


### Processing duplicates

#### 'children'

In [ ]:
#delete '-' and '0' from 'children' variable
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts().sort_values())

#### 'days_employed'

In [ ]:
# let's investigate 'days_employed' variable deeply. Particularly the nature of negative values.
# Build histogram to see wich destribution observatioins has
plt.hist(data['days_employed'], bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');
print()

In [ ]:
#It's seen that variable consist two groups of observations
#One group consist of 100% nembers less than zero. Values in range from 24.14 to 18388.95 days employed
#It highly possible that minuses it's only technical error, and we can easily cut them.
plt.hist(data['days_employed'][data['days_employed'] < 0], bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

print()
print('range: min - {:.2f}, max - {:.2f}'.format(
    data['days_employed'][data['days_employed'] < 0].abs().min(),
    data['days_employed'][data['days_employed'] < 0].abs().max()))

In [ ]:
#The second group consist values in range from from 328 728.72, to 401 755.40 days employed
plt.hist(data['days_employed'][data['days_employed'] > 0], bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');
print(stats.shapiro(data['days_employed'][data['days_employed'] > 0].abs()))
print()
print('range: min - {:.2f}, max - {:.2f}'.format(
    data['days_employed'][data['days_employed'] > 0].min(),
    data['days_employed'][data['days_employed'] > 0].max()))

In [ ]:
#Discovering the percent of observations in groups
print("Percent of observations in groups in variable 'days_emplyed': {:.2%} - <0; {:.2%} - 0; {:.2%} - >0".format(
    data['days_employed'][data['days_employed'] < 0].count() / data['days_employed'].count(),
    data['days_employed'][data['days_employed'] == 0].count() / data['days_employed'].count(),
    data['days_employed'][data['days_employed'] > 0].count() / data['days_employed'].count()))
print()

#let's delete minus symbol as a result of a technical issue
data['days_employed'] = data['days_employed'].abs()

#### 'education'

In [ ]:
#reducing case to the low level for the all types of education
data['education'] = data['education'].str.lower()
data['education'].value_counts()

### Conclusion

To testing data for an errors or duplicates I used ".value_counts()" method. It's good function for variables which consist not too much values. I find that 'children', 'education' and 'days_employed' data consists some mistakes. It's highly possible, that in 'children' and 'education' vars has a human factor error - cases-sensitively errors and extra symbols. 
For investigation of the negative values for 'days_employed' variable I decided to check if data in the var consistent or not. For this I built histogram, and surprisely discovered two set of data. And, expectedly, one of the sets was with negative values. It's point to the some systematically emerged error. So I decided to delete extra symbol "-" for all set by applying function "abs()". 

### Categorizing Data

#### 'days_employed'

In [ ]:
#divide the data in var 'days_employed' into three groups 'youths' 'veterans' and 'NA'
#values for separetion:
#print(data['days_employed'][data['days_employed'] < 0].abs().max())
#print(data['days_employed'][data['days_employed'] > 0].abs().min())

def employed_fun(day):
    if 0 < day < 18390:
        return 'youths'
    if day > 328727:
        return 'veterans'
    if day == 0:
        return 'NA'
    return 'Not a number'
    
data['employed_satus'] = data['days_employed'].apply(employed_fun)
print(data['employed_satus'].value_counts())

#### 'total_income'

In [ ]:
#lets chek var 'total_income' of clients
plt.hist(data['total_income'], bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');
#as seen destribution doesn't shown any visible groups

In [ ]:
#lets look how much observations in in approximate groups. I used an average salary 50k in SPb, Russia
print(data[data['total_income'] < 50000]['total_income'].value_counts(bins=3))
print(data[data['total_income'] > 50000]['total_income'].value_counts(bins=3))

#building a function for creating a new grouping variable 'income_group'
def income_fun(amount):
    if amount <= 20000:
        return "poor"
    if 20000 < amount <= 40000:
        return "lower_middle"
    if 40000 < amount <= 65000:
        return "middle"
    if 65000 < amount:
        return "higher_middle"
    return "NA"

#applying funtion to the data            
data['income_group'] = data['total_income'].apply(income_fun)
print(data['income_group'].value_counts())

#### 'income_type'

In [ ]:
#let's combine the meaning of “entrepreneur” on “business”
data.loc[data['income_type'] == 'entrepreneur', 'income_type'] = 'business'
#Delete insignificant "unemployed", "paternity / maternity leave" and "student"
data = data.drop(data[data.income_type == "unemployed"].index)
data = data.drop(data[data.income_type == "paternity / maternity leave"].index)
data = data.drop(data[data.income_type == "student"].index)

print(data['income_type'].value_counts())

#### 'gender'

In [ ]:
data = data.drop(data[data.gender == "XNA"].index)
data['gender'].value_counts()

#### 'purpose'

In [ ]:
#look to the variable 'purpse' closly and find counting by meaning values
data['purpose'].value_counts()

#As seen there are many. Let's getered output in a list for stemming
purpose_dict = dict(data['purpose'].value_counts())
purpose_names = []
for name in purpose_dict:
    purpose_names.append(name)

print(purpose_names)
print()
print()

# Creating a list of most common stems from the previously created list of proposes
import nltk
from nltk.stem import SnowballStemmer

stemmed_word = []
english_stemmer = SnowballStemmer('english')
for purpose in purpose_names:
    for word in purpose.split():
        stemmed_word.append(english_stemmer.stem(word))
        
from collections import Counter
print(Counter(stemmed_word).most_common())

In [ ]:
#building a function for creating a new grouping variable 'purpose_group' based on var 'purpose'
def purpose_fun(purpose_names):
    for word in purpose_names.split():
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'car':
            return "car" 
        if stemmed_word == 'educ':
            return "education"
        if stemmed_word == 'univers':
            return "education"
        if stemmed_word == 'transact':
            return "commercial"
        if stemmed_word == 'rent':
            return "commercial"
        if stemmed_word == 'commerci':
            return "commercial"
        if stemmed_word == 'wed':
            return "wedding"

#applying funtion to the data            
data['purpose_group'] = data['purpose'].apply(purpose_fun)
data['purpose_group'] = data['purpose_group'].fillna(value='real estate')

#Сheking for NA's
print(data['purpose'][data['purpose_group'].isna()].value_counts())
print()

#looking how much observations per any variable
print(data['purpose_group'].value_counts())
print()

#Checking the correct distribution of values into groups 
data_pivot = data.pivot_table(
index=['purpose_group', 'purpose'],
values='total_income', 
aggfunc='count')
print(data_pivot)
print()

#data2.groupby('purpose_group')['purpose'].count()

In [ ]:
#let's look if everything fine
data.info()
data.isna().sum()

### Conclusion

On the previous step I found two groups inside value **'days_employed'**. Then I decided to split it into three groups *'youths'* *'veterans'* and *'NA'*. Group 'NA' is equal to the zero. I used this name because zero means absent of the information.
I discovered **'total_income'** by using histogram but as seen distribution doesn't show any visible groups. But further analysis required to divide this variable to the groups. I decided to create a groups, grounded on expert knowledge about minimum income, average and high salary. Last affected var was **'gender'**, I cut out only one value, because it couldn't give us significant information. In the last variable **'purpose'** was seen a lot of different options of purposes. It seems that they need to be unify in larger groups. First I found grouping values by using stem method. It's easiest then lennomination and good enough for this task. So I revealed: *wedding* - wed; *commercial* - transact, rent, commerci; *car* - car; *education* - educ, univers; *real estate* - for the rest. By creating new function, I made a new variable. It wasn't easy to redistribute values of **'purpose'** among grouping values. Because some words are found in several values at once. Pivot table helped me in solving this problem. 

### Step 3. Major questions of the research

### - Is there a relation between having kids and repaying a loan on time?

In [ ]:
data_pivot = data.pivot_table(
index=['children'],
columns='debt',
values='family_status_id', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

Families without children and with three children repay debts better. But in a group with three children, there are too few observations and they may be insignificant. As a conclusion - families with children repay debts worse.

### - Is there a relation between marital status and repaying a loan on time?

In [ ]:
data_pivot = data.pivot_table(
index=['family_status'],
columns='debt',
values='family_status_id', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

Unmarried and divorced repaying loans worse than married. The best payers are divorced and widowers / widows

### - Is there a relation between income level and repaying a loan on time?

In [ ]:
data_pivot = data.pivot_table(
index=['income_group'],
columns='debt',
values='family_status_id', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

In [ ]:
# let's try to rebuild our groups to know if we could get data with another proportion of repay a loan
def income_fun(amount):
    if amount <= 20000:
        return "poor"
    if 20000 < amount <= 40000:
        return "lower_middle"
    if 40000 < amount <= 125000:
        return "middle"
    if 125000 < amount:
        return "higher_middle"
    return "NA"

#applying funtion to the data            
data['income_group'] = data['total_income'].apply(income_fun)
print(data['income_group'].value_counts())

In [ ]:
#build another data pivot
data_pivot = data.pivot_table(
index=['income_group'],
columns='debt',
values='family_status_id', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

Observed interesting results. First data groping variant showed as expected, that the richer clients the better chance to repay a loan. But after shifting upper limit, ratio of probability changed for rich people. It shows that people with high income worse repay loan the poorest clients. Only one important think require high attention. The total amount of people in 'higher_middle' group is 44. It's not enough for comparison with another large groups. This conclusion has to be verified on larger data. 

### - How do different loan purposes affect on-time repayment of the loan?

In [ ]:
data_pivot = data.pivot_table(
index=['purpose_group'],
columns='debt',
values='total_income', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

A loan for real estate, both for life and business, is better repaid than for a car or education. Interestingly, the wedding loans only a little worse repaid than real estate loans.

### - Is there education level affect on-time repayment of the loan?

In [ ]:
data_pivot = data.pivot_table(
index=['education'],
columns='debt',
values='total_income', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

Very interesting and seems significant results getting from education data of the clients. The educated clients the better chance to repay loan.

### - Is there a relation between employed status and repaying a loan?

In [ ]:
data_pivot = data.pivot_table(
index=['employed_satus'],
columns='debt',
values='total_income', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

Also interesting and significant results. People who have dramatically more work experience days - starting from 328727 days - has much more chances to repay loan.

### - Is there a relation between the customer’s income type and repaying a loan?

In [ ]:
data_pivot = data.pivot_table(
index=['income_type'],
columns='debt',
values='total_income', 
aggfunc='count')

data_pivot['ratio'] = data_pivot[1]/data_pivot[0]*100
print(data_pivot.sort_values(by='ratio', ascending=False))

### Conclusion

As seen from the data, people on retirement better repaying loan, as much as civil servants. Business repays loans little worse. The worst loan payers is employes.

### Step 4. General conclusion

Has been analyzed large amount of data consists information about clients who have some relations with bank debt. The data provided had some errors and inaccuracies which required to deal with them. I made data suitable for analysis by applying a number of functions. 

Here some preliminary results. Can be primarily designated further hypothesis according to the chances to repay a loan:

1. existence of the children decries chances to repay a loan
2. (marriage/divorced/widow(er)) status is increase chances to repay a loan;
3. rich and middle class repay loans better the poor and lower-middle class;
4. loan for auto and education repaying worse, than loans for wedding, commers or real estate costs;
5. the educated clients the better chance to repay loan;
6. clients with more experience has more chances to repay loan;
7. retired, servants and business is better loan payers than employes.

Import to remember that results don’t show causation only some correlation between variables. For discovered deeper results, with structural relations and causations among data, more serious statistical analysis required.